In [ ]:
!pip install mistralai

In [ ]:
import pandas as pd
from mistralai import Mistral
from google.colab import userdata

api = userdata.get('MISTRAL_API_KEY')

client = Mistral(api_key=api)

In [ ]:
system_message = """
You are a cybersecurity expert trained to analyze network flow logs and determine whether the traffic is benign or part of a Distributed Denial of Service (DDoS) attack.

Use the ReAct (Reasoning + Acting) approach:
1. First, REASON through the meaning of the network features such as IPs, ports, protocols, flow duration, packet counts, and byte rates.
2. Then, ACT by concluding whether the flow is BENIGN or DDOS based on patterns, anomalies, or suspicious behavior.

Be concise but clear in your reasoning, and always end with the final classification in the format: "Final Answer: BENIGN" or "Final Answer: DDOS".
"""

In [ ]:
df = pd.read_csv('/content/sample_20_rows.csv')
X = df.drop('Label', axis=1)
y = df['Label']

In [ ]:
for i in range(20):
    prompt = f"""
Source IP : {X['Source IP'][i]}
Destination IP : {X['Destination IP'][i]}
Source Port : {X['Source Port'][i]}
Destination Port : {X['Destination Port'][i]}
Protocol : {X['Protocol'][i]}
Flow Duration : {X['Flow Duration'][i]}
Total Fwd Packets : {X['Total Fwd Packets'][i]}
Total Backward Packets : {X['Total Backward Packets'][i]}
Flow Bytes/s : {X['Flow Bytes/s'][i]}
"""

    response = client.chat.complete(
        model="mistral-large-latest",
        messages=[
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.5,
        top_p=0.5,
        stream=False
    )

    print(f"\nResponse for index {i}:")
    print(response.choices[0].message.content)



Response for index 0:
### Reasoning:

1. **IP Addresses**:
   - Source IP (172.16.0.5) and Destination IP (192.168.50.1) are both private IP addresses, which is typical for internal network traffic.

2. **Ports**:
   - Source Port (842) is an ephemeral port, commonly used for outgoing connections.
   - Destination Port (21484) is also an ephemeral port, which is unusual for typical services but could be used for peer-to-peer applications or custom services.

3. **Protocol**:
   - Protocol 17 corresponds to UDP (User Datagram Protocol), which is often used for streaming audio, video, and VoIP, but also commonly used in DDoS attacks due to its connectionless nature.

4. **Flow Duration**:
   - Flow Duration is negative (-0.1944709240047926), which is unusual and suggests an error in data collection or a very short-lived flow.

5. **Packet Counts**:
   - Total Fwd Packets (0.3251184533735609) and Total Backward Packets (-0.1451238114943991) are both very low and negative values are suspi